<a href="https://colab.research.google.com/github/sayanbanerjee32/learning-equality-curriculum-recommendations/blob/main/lecr_XLMR_classification_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
is_colab = not os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [2]:
# install fastkaggle if not available
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle
## install collab specific libraries
if is_colab:
    !pip install -Uq datasets
    !pip install -Uq transformers
    !pip install -Uq sentencepiece 
    !pip install -Uq pynvml

!pip install -Uq evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from fastkaggle import *
from pathlib import Path
import pandas as pd
import numpy as np # linear algebra
from tqdm.auto import tqdm
import gc
from functools import partial

import torch
from transformers import (AutoModelForSequenceClassification,
                          AutoTokenizer, 
                          DataCollatorWithPadding, 
                          TrainingArguments,
                          Trainer, 
                          IntervalStrategy)
from datasets import Dataset, DatasetDict, ClassLabel
import evaluate

device = "cuda" if torch.cuda.is_available() else "cpu"
%env TOKENIZERS_PARALLELISM=true

env: TOKENIZERS_PARALLELISM=true


In [4]:
# config depending on whether this is running on kaggle or collab
# is_colab = True
comp = 'learning-equality-curriculum-recommendations'
if is_colab:
    model_save_path = Path('/content/drive/MyDrive/Colab Notebooks/models/'+comp)
    data_save_path = Path('/content/drive/MyDrive/Colab Notebooks/data/'+comp)
else:
    model_save_path = Path('/kaggle/working/'+comp) #+'/models')
    data_save_path = Path('/kaggle/input/lecr-sentence-transformer-data-prep-baseline/' + comp)

In [5]:
# import colab libraries
if is_colab:
    from google.colab import output
    output.enable_custom_widget_manager()
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so lets move it there.
if is_colab:
    !mkdir ~/.kaggle
    !cp /content/drive/MyDrive/Kaggle_api_auth/kaggle.json ~/.kaggle/

In [7]:
# This permissions change avoids a warning on Kaggle tool startup.
if is_colab:
    !chmod 600 ~/.kaggle/kaggle.json

In [8]:
path = setup_comp(comp)
path

100%|██████████| 254M/254M [00:11<00:00, 22.6MB/s]


Path('learning-equality-curriculum-recommendations')

### read all input files

In [9]:
df = pd.read_pickle(data_save_path / 'topic_content_details.pkl')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129115 entries, 0 to 129114
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   topic_id             129115 non-null  object
 1   content_id           129115 non-null  object
 2   label                129115 non-null  int64 
 3   title_topic          129115 non-null  object
 4   description_topic    129115 non-null  object
 5   channel              129115 non-null  object
 6   category             129115 non-null  object
 7   level                129115 non-null  int64 
 8   language_topic       129115 non-null  object
 9   parent               129115 non-null  object
 10  has_content          129115 non-null  bool  
 11  title_content        129115 non-null  object
 12  description_content  129115 non-null  object
 13  kind                 129115 non-null  object
 14  text                 129115 non-null  object
 15  language_content     129115 non-nu

In [10]:
## all has contents - no breadcrums yet
df.has_content.value_counts()

True    129115
Name: has_content, dtype: int64

In [11]:
### select only the ones that have same language
df = df.loc[df.language_content == df.language_topic]
df.shape

(112154, 18)

In [12]:
# random sampling to test in collab
if is_colab:
    df = df.sample(frac=0.10)
else: df = df.sample(frac=0.40)
df.shape[0]

11215

In [13]:
### only required columns
req_col = ['text','title_content','description_content','title_topic','description_topic','label']
ds = Dataset.from_pandas(df[req_col])
ds

Dataset({
    features: ['text', 'title_content', 'description_content', 'title_topic', 'description_topic', 'label', '__index_level_0__'],
    num_rows: 11215
})

In [14]:
del(df)
gc.collect()

16

In [15]:
# pre -process content text
# df['masked_content_txt'] = df[['title_content','text']].apply(lambda row: row.text.strip().replace(row.title_content.strip(),
#                                                                                                    '__MASKED__').strip().split("\n")[0][:100],
#                                                                axis = 1)
def clean_content_text(examples):
        return {
        'masked_content_txt': (
                                examples['text']#.strip()
                               .replace(examples['title_content'].strip(), '__MASKED__').strip()
                               .split("\n")[0][:100]
        )
    }

ds = ds.map(clean_content_text)

Map:   0%|          | 0/11215 [00:00<?, ? examples/s]

In [16]:
# df['input'] =  "CONTENT: " + (df["title_content"]
#         + " \n "
#         + df["description_content"].str.strip()
#         + " \n "
#         + df["masked_content_txt"]).str.strip() + " TOPIC: " + (df["title_topic"]
#         + " \n "
#         + df["description_topic"].str.strip()
#         ).str.strip()
def create_input(examples):
    return {
                'input': "CONTENT: " + (examples["title_content"]
                + " \n "
                + examples["description_content"].strip()
                + " \n "
                + examples["masked_content_txt"]).strip() 
                + " \n "
                + "TOPIC: " + (examples["title_topic"]
                + " \n "
                + examples["description_topic"].strip()
                ).strip()
}

ds = ds.map(create_input)
# df['seq_length_input'] = [len(txt.split()) for txt in df['input'].tolist()]
# df['seq_length_input'].describe()

Map:   0%|          | 0/11215 [00:00<?, ? examples/s]

In [18]:
# label encoding
labels = ClassLabel(names=list(set(ds['label'])))
labels

ClassLabel(names=[0, 1], id=None)

In [19]:
model_nm = 'xlm-roberta-base'
# 'bert-base-multilingual-cased'

In [20]:
tokz = AutoTokenizer.from_pretrained(model_nm)
tokz.model_max_length, tokz.is_fast

(512, True)

In [21]:
def tok_func(batch, is_test = False): 
    tokens = tokz(batch["input"], padding="longest", truncation=True)
    if not is_test:
        tokens['labels'] = labels.str2int(batch['label'])
        # [float(l) for l in labels.str2int(batch['discourse_effectiveness'])]
    return tokens
tok_ds = ds.map(tok_func, batched=True, batch_size = 16)

Map:   0%|          | 0/11215 [00:00<?, ? examples/s]

In [22]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['text', 'title_content', 'description_content', 'title_topic', 'description_topic', 'label', '__index_level_0__', 'masked_content_txt', 'input', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 8411
    })
    test: Dataset({
        features: ['text', 'title_content', 'description_content', 'title_topic', 'description_topic', 'label', '__index_level_0__', 'masked_content_txt', 'input', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2804
    })
})

In [23]:
data_collator = DataCollatorWithPadding(tokenizer=tokz)

In [24]:
bs = 16
grad_acc = 16
epochs = 1
lr = 5e-5
metric_name = "f1"

args = TrainingArguments('outputs',
                         learning_rate=lr,
                         warmup_ratio=0.1,
                         lr_scheduler_type='cosine',
                         fp16=True,
                        evaluation_strategy="epoch",
                         logging_strategy = "epoch",
                        #  logging_steps = 10,
                        #  eval_steps = 10,
                         per_device_train_batch_size=bs,
                          per_device_eval_batch_size=bs*2,
                        num_train_epochs=epochs,
                          weight_decay=0.01,
                         gradient_accumulation_steps=grad_acc,
                        #   load_best_model_at_end=True,
                            metric_for_best_model=metric_name,
                        #  label_names = labels.names, # possibly this is messing up with metrics
                          report_to='none')

In [25]:
# helper function cleaning GPU memory
def report_gpu():
    if is_colab: print(torch.cuda.list_gpu_processes())
    gc.collect()
    torch.cuda.empty_cache()
report_gpu()

GPU:0
no processes are running


In [26]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, 
                                                           num_labels=labels.num_classes)
def acc_metrics(eval_preds):
    metric = evaluate.load("f1")
    # metric = evaluate.load("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions,
                          references=labels,
                          average = 'weighted')
    # return {"accuracy": (predictions == labels).astype(np.float32).mean().item()}

trainer = Trainer(model, args, 
                  train_dataset=dds['train'], 
                  eval_dataset=dds['test'],
                  data_collator=data_collator,
                  tokenizer=tokz,
                  compute_metrics=acc_metrics)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [ ]:
trainer.train();

The following columns in the training set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, title_topic, description_topic, input, description_content, masked_content_txt, title_content, text. If __index_level_0__, title_topic, description_topic, input, description_content, masked_content_txt, title_content, text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8411
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient A

Epoch,Training Loss,Validation Loss


In [ ]:
### dump model
model_save_path.mkdir(parents=True, exist_ok=True)
trainer.save_model(model_save_path)

In [ ]:
if not iskaggle:
    push_notebook('saan', comp,
                  title='LECR: XLMR - classification- baseline',
                  file='/content/drive/MyDrive/Colab Notebooks/lecr_XLMR_classification_baseline.ipynb',
                  competition=comp, private=False, gpu=True)